In [15]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel
import pyspark.sql 
from pyspark.sql import SQLContext
from pyspark.sql.functions import * 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType,FloatType
from pyspark.sql.types import Row


In [2]:
spark = SparkContext(master="local", appName="Proyecto")
sqlContext = SQLContext(spark)

In [3]:
!ls curso-apache-spark-platzi/proyecto/

athlete_events.csv  deportista2.csv  evento.csv  modelo_relacional.jpg
deporte.csv	    deportista.csv   juegos.csv  paises.csv


In [4]:
!ls /home/vagrant/curso-apache-spark-platzi/proyecto/

athlete_events.csv  deportista2.csv  evento.csv  modelo_relacional.jpg
deporte.csv	    deportista.csv   juegos.csv  paises.csv


In [104]:
!head -n 5 /home/vagrant/curso-apache-spark-platzi/proyecto/athlete_events.csv

resultado_id,medalla,deportista_id,juego_id,evento_id
1,NA,1,39,1
2,NA,2,49,2
3,NA,3,7,3
4,Gold,4,2,4


In [110]:
def dropFirstRow(index,iterator):
     return iter(list(iterator)[1:]) #if index==0 else iterator

from pyspark.sql.functions import udf
colsInt = udf(lambda z: toInt(z), IntegerType())
colsFloat = udf(lambda z: toFloat(z), FloatType())

sqlContext.udf.register("colsInt", colsInt)
sqlContext.udf.register("colsFloat", colsFloat)

def toInt(s):
    if isinstance(s, str) == True:
        st = [str(ord(i)) for i in s]
        return(int(''.join(st)))
    else:
         return Null
    
def toFloat(s):
    if isinstance(s, str) == True:
        st = [str(ord(i)) for i in s]
        return(float(''.join(st)))
    else:
         return Null



In [122]:
path = "/home/vagrant/curso-apache-spark-platzi/proyecto/"

deportistaOlimpicoRDD =  spark.textFile(path+"deportista.csv").map(lambda line : line.split(","))
deportistaOlimpico2RDD = spark.textFile(path+"deportista2.csv").map(lambda line : line.split(","))
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpico2RDD)

deportistaOlimpicoRDD=deportistaOlimpicoRDD.mapPartitionsWithIndex(dropFirstRow)

deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l : (
toInt(l[0]),
l[1],
int(l[2]),
int(l[3]),
int(l[4]),
float(l[5]),
int(l[6])
))

schema = StructType([
StructField("deportista_id",IntegerType(),False)     ,
StructField("nombre",StringType(),False)        ,
StructField("genero",IntegerType(),False)        ,
StructField("edad",IntegerType(),True)      ,
StructField("altura",IntegerType(),True)        ,
StructField("peso",FloatType(),True)      ,
StructField("equipo_id",IntegerType(),True)     
])

deportesOlimpicosDF = sqlContext.createDataFrame(deportistaOlimpicoRDD,schema)


In [123]:
deportesOlimpicosDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- genero: integer (nullable = false)
 |-- edad: integer (nullable = true)
 |-- altura: integer (nullable = true)
 |-- peso: float (nullable = true)
 |-- equipo_id: integer (nullable = true)



In [112]:
paisesRDD = spark.textFile(path+"paises.csv",2).map(lambda line : line.split(","))
paisesRDD = paisesRDD.mapPartitionsWithIndex(dropFirstRow)

paisesRDD = paisesRDD.map(lambda l : (
int(l[0]),
l[1],
l[2]
))

schema = StructType([
StructField("id",IntegerType(),False),
StructField("equipo",StringType(),False),
StructField("sigla",StringType(),False)
])

paisesDF = sqlContext.createDataFrame(paisesRDD,schema)

paisesDF.take(5)

[Row(id=1, equipo='30. Februar', sigla='AUT'),
 Row(id=2, equipo='A North American Team', sigla='MEX'),
 Row(id=3, equipo='Acipactli', sigla='MEX'),
 Row(id=4, equipo='Acturus', sigla='ARG'),
 Row(id=5, equipo='Afghanistan', sigla='AFG')]

In [124]:
deportesOlimpicosRDDSchema = StructType(
    [StructField("deporte_id",IntegerType(),False),
     StructField("deporte",StringType(),False)
    ])

deportesOlimpicosDF = sqlContext.read.schema(deportesOlimpicosRDDSchema).option("header","true").csv(path+"deporte.csv")
deportesOlimpicosDF.take(6)

[Row(deporte_id=1, deporte='Basketball'),
 Row(deporte_id=2, deporte='Judo'),
 Row(deporte_id=3, deporte='Football'),
 Row(deporte_id=4, deporte='Tug-Of-War'),
 Row(deporte_id=5, deporte='Speed Skating'),
 Row(deporte_id=6, deporte='Cross Country Skiing')]

In [126]:
eventoSchema= StructType([
    StructField("evento_id",IntegerType(),False),
    StructField("nombre",StringType(),False),
    StructField("deporte_id",IntegerType(),False)
])

deportesOlimpicosDF = sqlContext.read.schema(eventoSchema).option("header","true").csv(path+"evento.csv")
deportesOlimpicosDF.take(6)

[Row(evento_id=1, nombre="Basketball Men's Basketball", deporte_id=1),
 Row(evento_id=2, nombre="Judo Men's Extra-Lightweight", deporte_id=2),
 Row(evento_id=3, nombre="Football Men's Football", deporte_id=3),
 Row(evento_id=4, nombre="Tug-Of-War Men's Tug-Of-War", deporte_id=4),
 Row(evento_id=5, nombre="Speed Skating Women's 500 metres", deporte_id=5),
 Row(evento_id=6, nombre="Speed Skating Women's 1,000 metres", deporte_id=5)]

In [39]:

juegoSchema = StructType([
    StructField("juego_id",IntegerType(),False),
    StructField("anio",StringType(),False),
    StructField("temporada",StringType(),False),
    StructField("ciudad",StringType(),False),
])
juegoDF = sqlContext.read.schema(juegoSchema).option("header","true").csv(path+"juegos.csv")

resultadoSchema = StructType([
    StructField("resultado_id",IntegerType(),False),
    StructField("medalla",StringType(),False),
    StructField("deporitsta_id",IntegerType(),False),
    StructField("juego_id",IntegerType(),False),
    StructField("evento_id",IntegerType(),False),
])
resultadoDF = sqlContext.read.schema(resultadoSchema).option("header","true").csv(path+"athlete_events.csv")


In [40]:
juegoDF.take(5)

[Row(juego_id=1, anio='1896 Verano', temporada='1896', ciudad='Verano'),
 Row(juego_id=2, anio='1900 Verano', temporada='1900', ciudad='Verano'),
 Row(juego_id=3, anio='1904 Verano', temporada='1904', ciudad='Verano'),
 Row(juego_id=4, anio='1906 Verano', temporada='1906', ciudad='Verano'),
 Row(juego_id=5, anio='1908 Verano', temporada='1908', ciudad='Verano')]

In [41]:
resultadoDF.take(5)

[Row(resultado_id=1, medalla='NA', deporitsta_id=1, juego_id=39, evento_id=1),
 Row(resultado_id=2, medalla='NA', deporitsta_id=2, juego_id=49, evento_id=2),
 Row(resultado_id=3, medalla='NA', deporitsta_id=3, juego_id=7, evento_id=3),
 Row(resultado_id=4, medalla='Gold', deporitsta_id=4, juego_id=2, evento_id=4),
 Row(resultado_id=5, medalla='NA', deporitsta_id=5, juego_id=36, evento_id=5)]

In [47]:
deportesOlimpicosDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- genero: integer (nullable = false)
 |-- edad: integer (nullable = true)
 |-- altura: integer (nullable = true)
 |-- peso: float (nullable = true)
 |-- equipo_id: integer (nullable = true)



# Operaciones sobre DF

In [71]:
colNombre = deportesOlimpicosDF["nombre"]
colAltura = deportesOlimpicosDF.altura

deportesOlimpicosDF.select(colNombre).collect()

Py4JJavaError: An error occurred while calling o1585.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 41.0 failed 1 times, most recent failure: Lost task 0.0 in stage 41.0 (TID 47, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/vagrant/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/vagrant/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/vagrant/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/vagrant/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-46-88025a4d40b7>", line 16, in <lambda>
ValueError: invalid literal for int() with base 10: ''

	at org.a

In [54]:
deportesOlimpicosDF.where(colAltura > 160).take(10)

[Row(deportista_id=1, nombre='A Dijiang', genero=1, edad=24, altura=180, peso=80.0, equipo_id=199),
 Row(deportista_id=2, nombre='A Lamusi', genero=1, edad=23, altura=170, peso=60.0, equipo_id=199),
 Row(deportista_id=5, nombre='Christine Jacoba Aaftink', genero=2, edad=21, altura=185, peso=82.0, equipo_id=705),
 Row(deportista_id=6, nombre='Per Knut Aaland', genero=1, edad=31, altura=188, peso=75.0, equipo_id=1096),
 Row(deportista_id=7, nombre='John Aalberg', genero=1, edad=31, altura=183, peso=72.0, equipo_id=1096),
 Row(deportista_id=8, nombre='"Cornelia ""Cor"" Aalten (-Strannood)"', genero=2, edad=18, altura=168, peso=0.0, equipo_id=705),
 Row(deportista_id=9, nombre='Antti Sami Aalto', genero=1, edad=26, altura=186, peso=96.0, equipo_id=350),
 Row(deportista_id=11, nombre='Jorma Ilmari Aalto', genero=1, edad=22, altura=182, peso=76.5, equipo_id=350),
 Row(deportista_id=12, nombre='Jyri Tapani Aalto', genero=1, edad=31, altura=172, peso=70.0, equipo_id=350),
 Row(deportista_id=14

In [102]:
deportesOlimpicosDF.filter("equipo_id" == 50).show(truncate=False)

TypeError: condition should be string or Column

In [45]:
goldMedals = resultadoDF.filter(resultadoDF.medalla == 'Gold').join(deportesOlimpicosDF, 
                                            deportesOlimpicosDF.deportista_id == resultadoDF.deporitsta_id,
                                            "left").join(equiposOlimipicosDF,
                                            equiposOlimipicosDF.um     == deportesOlimpicosDF.equipo_id,
                                            "left").select("medalla","nombre","Team","NOC")


AttributeError: 'DataFrame' object has no attribute 'deportista_id'

In [ ]:
goldMedals = goldMedals.select(col("medalla"),
                               col("nombre"),
                               col("Team").alias("equipo"),
                               col("NOC").alias("CO"))

In [ ]:
goldMedals.groupBy("CO").count().sort(desc("count")).take(10)

In [ ]:
deportistaMedallas = resultadoDF.filter((resultadoDF.medalla == 'Gold') |
                   (resultadoDF.medalla == 'Silver') |
                   (resultadoDF.medalla == 'Bronze')).join(
                        deportistaDF, deportistaDF.deportista_id == resultadoDF.deporitsta_id,"left").select(
                        "medalla","nombre")

In [ ]:
top10Medallas = deportistaMedallas.groupBy("nombre").count().sort(desc("count")).take(10)

In [ ]:
!pwd

In [ ]:
sqlContext.createDataFrame(top10Medallas).write.format('csv').option('header',True).mode('overwrite').option("sep",",").save("top10Medallas.csv")

In [ ]:
#spark.stop()